In [33]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [34]:
from ETL.extract.extract import Extract
extract_class = Extract()

Data sales_first

In [35]:
path_file = 'Data_sales_source/ElectronicsProductsPricingData.csv'


In [36]:
df_sales = pd.read_csv(path_file)

In [37]:
df_sales.head(3)

,id,prices.amountMax,prices.amountMin,prices.availability,prices.condition,prices.currency,prices.dateSeen,prices.isSale,prices.merchant,prices.shipping,...,name,primaryCategories,sourceURLs,upc,weight,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30
0,AVphzgbJLJeJML43fA0o,104.99,104.99,Yes,New,USD,"2017-03-30T06:00:00Z,2017-03-10T22:00:00Z,2017...",False,Bestbuy.com,NaN,...,Sanus VLF410B1 10-Inch Super Slim Full-Motion ...,Electronics,https://www.amazon.com/Sanus-VLF410B1-10-Inch-...,7.94E+11,32.8 pounds,NaN,NaN,NaN,NaN,NaN
1,AVpgMuGwLJeJML43KY_c,69.00,64.99,In Stock,New,USD,2017-12-14T06:00:00Z,True,Walmart.com,Expedited,...,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,http://reviews.bestbuy.com/3545/4784804/review...,6.42E+11,14 pounds,NaN,NaN,NaN,NaN,NaN
2,AVpgMuGwLJeJML43KY_c,69.00,69.00,In Stock,New,USD,2017-09-08T05:00:00Z,False,Walmart.com,Expedited,...,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,http://reviews.bestbuy.com/3545/4784804/review...,6.42E+11,14 pounds,NaN,NaN,NaN,NaN,NaN


In [38]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7249 entries, 0 to 7248
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   7249 non-null   object 
 1   prices.amountMax     7249 non-null   float64
 2   prices.amountMin     7249 non-null   float64
 3   prices.availability  7249 non-null   object 
 4   prices.condition     7249 non-null   object 
 5   prices.currency      7249 non-null   object 
 6   prices.dateSeen      7249 non-null   object 
 7   prices.isSale        7249 non-null   bool   
 8   prices.merchant      7249 non-null   object 
 9   prices.shipping      4277 non-null   object 
 10  prices.sourceURLs    7249 non-null   object 
 11  asins                7249 non-null   object 
 12  brand                7249 non-null   object 
 13  categories           7249 non-null   object 
 14  dateAdded            7249 non-null   object 
 15  dateUpdated          7249 non-null   o

In [39]:
# cheking null data

df = df_sales.isna().sum().reset_index()
df.columns = ['col', "null_data"]

cond = df['null_data'] > 0
df_null = df[cond]
df_null['null_percentage'] = (df['null_data'] / len(df_sales) )* 100
df_null

/tmp/ipykernel_17412/3303632961.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_null['null_percentage'] = (df['null_data'] / len(df_sales) )* 100


,col,null_data,null_percentage
9,prices.shipping,2972,40.998758
16,ean,5706,78.714305
19,manufacturer,4014,55.373155
26,Unnamed: 26,7210,99.461995
27,Unnamed: 27,7231,99.751690
28,Unnamed: 28,7243,99.917230
29,Unnamed: 29,7237,99.834460
30,Unnamed: 30,7243,99.917230


In [40]:
# comple data null

data = df_sales[['prices.amountMin', 'prices.amountMax', 'prices.shipping', 'manufacturer', 'name']]


In [41]:
data_price_shipping = data[['prices.amountMin', 'prices.amountMax', 'prices.shipping']]
data_price_shipping = data_price_shipping[data_price_shipping['prices.shipping'].isna()]
df_High_45 = data_price_shipping[data_price_shipping['prices.amountMax'] > 45]
df_Low_45 = data_price_shipping[data_price_shipping['prices.amountMax'] < 45]

In [42]:
df_Low_45[df_Low_45['prices.amountMin'] < 35]

,prices.amountMin,prices.amountMax,prices.shipping
13,23.99,23.99,NaN
14,31.99,31.99,NaN
16,34.99,34.99,NaN
17,23.99,23.99,NaN
96,22.99,22.99,NaN
...,...,...,...
7167,22.99,22.99,NaN
7168,28.99,28.99,NaN
7170,26.99,26.99,NaN
7173,19.99,19.99,NaN


untuk data price shipping dilakukan mapping berdasarkan Peromo
Free Expedited Shipping for most orders over $49
→ Gratis jika pembelian di atas $49

Free Standard Shipping on Orders Over $49
→ Gratis jika pembelian di atas $49

Free Shipping on orders 35 and up
→ Gratis jika pembelian di atas $35

jika masih ada yang null setelah di filter 3 promo di atas akan di drop jika kurang dari 10% data yang null

setelah itu akan di transfrom menjadi int agar bisa di hitung

In [43]:
def check_null( df):
    df_null = df.isnull().sum().reset_index()
    df_null.columns = ['col', 'missing_val']
    df_null_col = df_null[df_null['missing_val'] > 0].copy()
    df_null_col['missing_val'] = df_null_col['missing_val'].astype(int)
    df_null_col['missing_value_percentage'] = round((df_null_col['missing_val'] / len(df)) * 100,2)
    return df_null_col

In [44]:
df_1 = check_null(df_sales)
df_1 = df_1[df_1['missing_value_percentage'] > 60]
col = list(df_1.col)
col

['ean',
 'Unnamed: 26',
 'Unnamed: 27',
 'Unnamed: 28',
 'Unnamed: 29',
 'Unnamed: 30']

In [45]:
def drop_columns_null(df,col):
    df_clear = df.drop(columns=col)
    return df_clear

In [46]:
df_dropped_null = drop_columns_null(df_sales, col)

In [47]:
def fill_shipping(row, row_1, row_2):
    if pd.isna(row[row_1]) and row[row_2] > 45:
        return 'Free Expedited Shipping for most orders over $49'
    elif pd.isna(row[row_1]) and row[row_2] > 35:
        return 'Free Expedited Shipping for most orders over $35'
    else:
        return row[row_1]



In [48]:

df_dropped_null['prices.shipping'] = df_dropped_null.apply(
    fill_shipping,
    axis=1,
    args=('prices.shipping', 'prices.amountMin')  
)

In [49]:
check_null(df_dropped_null)

,col,missing_val,missing_value_percentage
9,prices.shipping,203,2.80
18,manufacturer,4014,55.37


In [50]:
def fill_null(row):
    if pd.isna(row['manufacturer']):
        return row['brand']
    else:
        return row['manufacturer']

df_dropped_null['manufacturer'] = df_dropped_null.apply(fill_null, axis=1)


In [51]:
df_dropped_null['manufacturer'] = df_dropped_null['manufacturer'].fillna(df_dropped_null['brand'])

In [52]:
def drop_null_data(df):
    df_clean =  df.dropna()
    return df_clean

In [53]:
check_null(df_dropped_null)

,col,missing_val,missing_value_percentage
9,prices.shipping,203,2.8


In [54]:
df_clean = drop_null_data(df_dropped_null)

In [55]:
check_null(df_clean)

,col,missing_val,missing_value_percentage


In [56]:
def check_duplicated(df):
    df_cheked = df.duplicated().sum()
    return f'total data duplicate {df_cheked}'

In [57]:
check_duplicated(df_clean)

'total data duplicate 0'

In [58]:
# this is the clean data from sales_df
df_clean.head()

,id,prices.amountMax,prices.amountMin,prices.availability,prices.condition,prices.currency,prices.dateSeen,prices.isSale,prices.merchant,prices.shipping,...,dateUpdated,imageURLs,keys,manufacturer,manufacturerNumber,name,primaryCategories,sourceURLs,upc,weight
0,AVphzgbJLJeJML43fA0o,104.99,104.99,Yes,New,USD,"2017-03-30T06:00:00Z,2017-03-10T22:00:00Z,2017...",False,Bestbuy.com,Free Expedited Shipping for most orders over $49,...,2018-05-12T18:59:48Z,https://images-na.ssl-images-amazon.com/images...,sanusvlf410b110inchsuperslimfullmotionmountfor...,Sanus,VLF410B1,Sanus VLF410B1 10-Inch Super Slim Full-Motion ...,Electronics,https://www.amazon.com/Sanus-VLF410B1-10-Inch-...,7.94E+11,32.8 pounds
1,AVpgMuGwLJeJML43KY_c,69.00,64.99,In Stock,New,USD,2017-12-14T06:00:00Z,True,Walmart.com,Expedited,...,2018-06-13T19:39:02Z,https://images-na.ssl-images-amazon.com/images...,boytone2500w21chhometheatersystemblackdiamond/...,Boytone,BT-210F,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,http://reviews.bestbuy.com/3545/4784804/review...,6.42E+11,14 pounds
2,AVpgMuGwLJeJML43KY_c,69.00,69.00,In Stock,New,USD,2017-09-08T05:00:00Z,False,Walmart.com,Expedited,...,2018-06-13T19:39:02Z,https://images-na.ssl-images-amazon.com/images...,boytone2500w21chhometheatersystemblackdiamond/...,Boytone,BT-210F,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,http://reviews.bestbuy.com/3545/4784804/review...,6.42E+11,14 pounds
3,AVpgMuGwLJeJML43KY_c,69.99,69.99,Yes,New,USD,2017-10-10T05:00:00Z,False,Bestbuy.com,Free Expedited Shipping for most orders over $49,...,2018-06-13T19:39:02Z,https://images-na.ssl-images-amazon.com/images...,boytone2500w21chhometheatersystemblackdiamond/...,Boytone,BT-210F,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,http://reviews.bestbuy.com/3545/4784804/review...,6.42E+11,14 pounds
4,AVpgMuGwLJeJML43KY_c,66.99,66.99,Yes,New,USD,2017-08-28T07:00:00Z,False,Bestbuy.com,Free Expedited Shipping for most orders over $49,...,2018-06-13T19:39:02Z,https://images-na.ssl-images-amazon.com/images...,boytone2500w21chhometheatersystemblackdiamond/...,Boytone,BT-210F,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,http://reviews.bestbuy.com/3545/4784804/review...,6.42E+11,14 pounds


In [ ]:
def create_shipping_to_float(col):
    row_int = col
    if 'USD' in row_int:
        val_float = row_int.split(' ')[1]
        return float(val_float)
    else:
        return 0.0

In [ ]:
df_clean['shipping_cost'] = create_shipping_to_float(df_clean, 'prices.shipping')

/tmp/ipykernel_17412/1076417876.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['shipping_cost'] = create_shipping_to_float(df_clean, 'prices.shipping')


In [62]:
df_clean

,id,prices.amountMax,prices.amountMin,prices.availability,prices.condition,prices.currency,prices.dateSeen,prices.isSale,prices.merchant,prices.shipping,...,imageURLs,keys,manufacturer,manufacturerNumber,name,primaryCategories,sourceURLs,upc,weight,shipping_cost
0,AVphzgbJLJeJML43fA0o,104.99,104.99,Yes,New,USD,"2017-03-30T06:00:00Z,2017-03-10T22:00:00Z,2017...",False,Bestbuy.com,Free Expedited Shipping for most orders over $49,...,https://images-na.ssl-images-amazon.com/images...,sanusvlf410b110inchsuperslimfullmotionmountfor...,Sanus,VLF410B1,Sanus VLF410B1 10-Inch Super Slim Full-Motion ...,Electronics,https://www.amazon.com/Sanus-VLF410B1-10-Inch-...,7.94E+11,32.8 pounds,0.0
1,AVpgMuGwLJeJML43KY_c,69.00,64.99,In Stock,New,USD,2017-12-14T06:00:00Z,True,Walmart.com,Expedited,...,https://images-na.ssl-images-amazon.com/images...,boytone2500w21chhometheatersystemblackdiamond/...,Boytone,BT-210F,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,http://reviews.bestbuy.com/3545/4784804/review...,6.42E+11,14 pounds,0.0
2,AVpgMuGwLJeJML43KY_c,69.00,69.00,In Stock,New,USD,2017-09-08T05:00:00Z,False,Walmart.com,Expedited,...,https://images-na.ssl-images-amazon.com/images...,boytone2500w21chhometheatersystemblackdiamond/...,Boytone,BT-210F,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,http://reviews.bestbuy.com/3545/4784804/review...,6.42E+11,14 pounds,0.0
3,AVpgMuGwLJeJML43KY_c,69.99,69.99,Yes,New,USD,2017-10-10T05:00:00Z,False,Bestbuy.com,Free Expedited Shipping for most orders over $49,...,https://images-na.ssl-images-amazon.com/images...,boytone2500w21chhometheatersystemblackdiamond/...,Boytone,BT-210F,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,http://reviews.bestbuy.com/3545/4784804/review...,6.42E+11,14 pounds,0.0
4,AVpgMuGwLJeJML43KY_c,66.99,66.99,Yes,New,USD,2017-08-28T07:00:00Z,False,Bestbuy.com,Free Expedited Shipping for most orders over $49,...,https://images-na.ssl-images-amazon.com/images...,boytone2500w21chhometheatersystemblackdiamond/...,Boytone,BT-210F,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,http://reviews.bestbuy.com/3545/4784804/review...,6.42E+11,14 pounds,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7244,AVpfUsrxLJeJML437Ezd,79.99,79.99,Yes,New,USD,2017-10-10T08:00:00Z,False,Bestbuy.com,Free Expedited Shipping for most orders over $49,...,https://static.bhphoto.com/images/images500x50...,loweproslingshotedge250awcamerabackpackblack/5...,Lowepro,LP36899,Lowepro - Slingshot Edge 250 AW Camera Backpac...,Electronics,"http://www.frys.com/product/8595219,https://re...",56035368998,1.8 pounds,0.0
7245,AVpfUsrxLJeJML437Ezd,68.75,68.75,In Stock,New,USD,2018-06-01T12:00:00Z,False,bhphotovideo.com,Free Expedited Shipping for most orders over $49,...,https://static.bhphoto.com/images/images500x50...,loweproslingshotedge250awcamerabackpackblack/5...,Lowepro,LP36899,Lowepro - Slingshot Edge 250 AW Camera Backpac...,Electronics,"http://www.frys.com/product/8595219,https://re...",56035368998,1.8 pounds,0.0
7246,AVpfUsrxLJeJML437Ezd,70.99,70.99,Yes,New,USD,2017-05-18T05:00:00Z,False,Bestbuy.com,Free Expedited Shipping for most orders over $49,...,https://static.bhphoto.com/images/images500x50...,loweproslingshotedge250awcamerabackpackblack/5...,Lowepro,LP36899,Lowepro - Slingshot Edge 250 AW Camera Backpac...,Electronics,"http://www.frys.com/product/8595219,https://re...",56035368998,1.8 pounds,0.0
7247,AVpfUsrxLJeJML437Ezd,71.99,71.99,Yes,New,USD,2017-08-28T11:00:00Z,False,Bestbuy.com,Free Expedited Shipping for most orders over $49,...,https://static.bhphoto.com/images/images500x50...,loweproslingshotedge250awcamerabackpackblack/5...,Lowepro,LP36899,Lowepro - Slingshot Edge 250 AW Camera Backpac...,Electronics,"http://www.frys.com/product/8595219,https://re...",56035368998,1.8 pounds,0.0
